## Quickstart

### Introduction

Hi! Glad to see you made it to the Quickstart guide! In this guide you are introduced to how CARTOframes can be used by data scientists in spatial analysis workflows. Using simulated Starbucks revenue data, this guide walks through some common steps a data scientist takes to answer the following question: which stores are performing better than others?

Before you get started, we encourage you to have CARTOframes and Python 3 installed so you can get a feel for the library by using it:

In [ ]:
pip install cartoframes

For additional ways to install CARTOframes, check out the [Installation Guide](/developers/cartoframes/guides/Installation).

#### Spatial analysis scenario

Let's say you are a data scientist working for Starbucks and you want to better understand why some stores in Brooklyn, New York, perform better than others.

To begin, let's outline a workflow:

- Get and explore your company's data
- Create areas of influence for your stores
- Enrich your data with demographic data
- And finally, share the results of your analysis with your team

Let's get started!

### Get and explore your company's data

We are going to use a dataset that contains information about the location of Starbucks and each store's annual revenue.

As a first exploratory step, read it into a Jupyter Notebook using [pandas](https://pandas.pydata.org/).

In [1]:
import pandas as pd

stores_df = pd.read_csv('http://libs.cartocdn.com/cartoframes/files/starbucks_brooklyn.csv')
stores_df.head()

,name,address,revenue
0,Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772
1,607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418
2,65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699
3,Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676
4,Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411


To display your stores as points on a map, you first have to convert the `address` column into geometries. This process is called [geocoding](https://carto.com/help/working-with-data/geocoding-types/) and CARTO provides a straightforward way to do it (you can learn more about it in the [Location Data Services guide](/developers/cartoframes/guides/Location-Data-Services)).

In order to geocode, you have to set your CARTO credentials. If you aren't sure about your API key, check the [Authentication guide](/developers/cartoframes/guides/Authentication/) to learn how to get it. In case you want to see the geocoded result, without being logged in, [you can get it here](http://libs.cartocdn.com/cartoframes/files/starbucks_brooklyn_geocoded.csv).

> Note: If you don't have an account yet, you can get a trial, or a free account if you are a student, by [signing up here](https://carto.com/signup/).

In [2]:
from cartoframes.auth import set_default_credentials

set_default_credentials('creds.json')

Now that your credentials are set, we are ready to geocode the dataframe. The resulting data will be a [GeoDataFrame](http://geopandas.org/data_structures.html#geodataframe).

In [3]:
from cartoframes.data.services import Geocoding

stores_gdf, _ = Geocoding().geocode(stores_df, street='address')
stores_gdf.head()

Success! Data geocoded correctly


,the_geom,name,address,revenue,gc_status_rel,carto_geocode_hash
0,POINT (-73.93664 40.66930),Franklin Ave & Eastern Pkwy,"341 Eastern Pkwy,Brooklyn, NY 11238",1321040.772,0.91,9212e0e908d8c64d07c6a94827322397
1,POINT (-73.96122 40.57796),607 Brighton Beach Ave,"607 Brighton Beach Avenue,Brooklyn, NY 11235",1268080.418,0.97,b1bbfe2893914a350193969a682dc1f5
2,POINT (-73.98981 40.61950),65th St & 18th Ave,"6423 18th Avenue,Brooklyn, NY 11204",1248133.699,0.94,e47cf7b16d6c9b53c63e86a0418add1d
3,POINT (-74.02767 40.63204),Bay Ridge Pkwy & 3rd Ave,"7419 3rd Avenue,Brooklyn, NY 11209",1185702.676,0.94,2f21749c02f73116892eb3b6fd5d5738
4,POINT (-74.00098 40.59321),Caesar's Bay Shopping Center,"8973 Bay Parkway,Brooklyn, NY 11214",1148427.411,0.94,134c23973313802448365db6235783f9


Done! Now that the stores are geocoded, you will notice a new column named `geometry` has been added. This column stores the geographic location of each store and it's used to plot each location on the map.

You can quickly visualize your geocoded dataframe using the Map and Layer classes. Check out the [Visualization guide](/developers/cartoframes/guides/Visualization) to learn more about the visualization capabilities inside of CARTOframes.

In [4]:
from cartoframes.viz import Map, Layer

Map(Layer(stores_gdf))

Great! You have a map!

With the stores plotted on the map, you now have a better sense about where each one is. To continue your exploration, you want to know which stores earn the most yearly revenue. To do this, you can use the [`size_continuous_style`](/developers/cartoframes/examples/#example-size-continuous-style) visualization layer:

In [5]:
from cartoframes.viz import Map, Layer, size_continuous_style

Map(Layer(stores_gdf, size_continuous_style('revenue', size_range=[10,40]), title='Annual Revenue ($)'))

Good job! By using the [`size continuous visualization style`](/developers/cartoframes/examples/#example-size-continuous-style) you can see right away where the stores with higher revenue are. By default, visualization styles also provide a popup with the mapped value and an appropriate legend.

### Create your areas of influence

Similar to geocoding, there is a straightforward method for creating isochrones to define areas of influence around each store. Isochrones are concentric polygons that display equally calculated levels over a given surface area measured by time.

For this analysis, let's create isochrones for each store that cover the area within a 15 minute walk.

To do this you will use the Isolines data service:


In [6]:
from cartoframes.data.services import Isolines

isochrones_gdf, _ = Isolines().isochrones(stores_gdf, [15*60], mode='walk')
isochrones_gdf.head()

Success! Isolines created correctly


,source_id,data_range,the_geom
0,0,900,"MULTIPOLYGON (((-73.94683 40.67001, -73.94623 ..."
1,1,900,"MULTIPOLYGON (((-73.97017 40.57732, -73.96957 ..."
2,2,900,"MULTIPOLYGON (((-73.99781 40.62418, -73.99755 ..."
3,3,900,"MULTIPOLYGON (((-74.03678 40.63431, -74.03618 ..."
4,4,900,"MULTIPOLYGON (((-74.00451 40.59723, -74.00391 ..."


In [7]:
stores_map = Map([
    Layer(isochrones_gdf),
    Layer(stores_gdf, size_continuous_style('revenue', size_range=[10,40]), title='Annual Revenue ($)')
])

stores_map

There they are! To learn more about creating isochrones and isodistances check out the [Location Data Services guide](/developers/cartoframes/guides/Location-Data-Services).

> Note: You will see how to publish a map in the last section. If you already want to publish this map, you can do it by calling `stores_map.publish('starbucks_isochrones', password=None)`.

### Enrich your data with demographic data

Now that you have the area of influence calculated for each store, let's take a look at how to augment the result with population information to help better understand a store's average revenue per person.

> Note: To be able to use the Enrichment functions you need an enterprise CARTO account with Data Observatory 2.0 enabled. Contact your CSM or contact us at [sales@carto.com](mailto:sales@carto.com) for more information.


First, let's find the demographic variable we need. We will use the `Catalog` class that can be filter by country and category. In our case, we have to look for USA demographics datasets. Let's see which public ones are available.

In [8]:
from cartoframes.data.observatory import Catalog

datasets_df = Catalog().country('usa').category('demographics').datasets.to_dataframe()
datasets_df[datasets_df['is_public_data'] == True]

,category_id,category_name,country_id,data_source_id,description,geography_description,geography_id,geography_name,id,is_public_data,lang,name,provider_id,provider_name,slug,temporal_aggregation,time_coverage,update_frequency,version
0,demographics,Demographics,usa,sociodemographics,The American Community Survey (ACS) is an ongo...,Shoreline clipped TIGER/Line boundaries. More ...,carto-do-public-data.usa_carto.geography_usa_b...,Census Block Groups (2015) - shoreline clipped,carto-do-public-data.usa_acs.demographics_soci...,True,eng,5-yr ACS data at Census Block Groups level (20...,usa_acs,USA American Community Survey,acs_sociodemogr_b758e778,5yrs,"[2013-01-01,2018-01-01)",None,20132017
1,demographics,Demographics,usa,labor,Extract from the Quarterly Census of Emploment...,Shoreline clipped TIGER/Line boundaries. More ...,carto-do-public-data.usa_carto.geography_usa_c...,Counties (2018) - shoreline clipped,carto-do-public-data.usa_bls.demographics_labo...,True,eng,BLS QCEW data at county level (1999Q1),usa_bls,USA Bureau of Labor Statistics,bls_labor_399c6c81,quarterly,"[1999-01-01,1999-04-01)",None,1999Q1
2,demographics,Demographics,usa,labor,Extract from the Quarterly Census of Emploment...,Shoreline clipped TIGER/Line boundaries. More ...,carto-do-public-data.usa_carto.geography_usa_c...,Counties (2018) - shoreline clipped,carto-do-public-data.usa_bls.demographics_labo...,True,eng,BLS QCEW data at county level (1999Q2),usa_bls,USA Bureau of Labor Statistics,bls_labor_a0953d3b,quarterly,"[1999-04-01,1999-07-01)",None,1999Q2
3,demographics,Demographics,usa,labor,Extract from the Quarterly Census of Emploment...,Shoreline clipped TIGER/Line boundaries. More ...,carto-do-public-data.usa_carto.geography_usa_c...,Counties (2018) - shoreline clipped,carto-do-public-data.usa_bls.demographics_labo...,True,eng,BLS QCEW data at county level (1999Q3),usa_bls,USA Bureau of Labor Statistics,bls_labor_d7920dad,quarterly,"[1999-07-01,1999-10-01)",None,1999Q3
4,demographics,Demographics,usa,labor,Extract from the Quarterly Census of Emploment...,Shoreline clipped TIGER/Line boundaries. More ...,carto-do-public-data.usa_carto.geography_usa_c...,Counties (2018) - shoreline clipped,carto-do-public-data.usa_bls.demographics_labo...,True,eng,BLS QCEW data at county level (1999Q4),usa_bls,USA Bureau of Labor Statistics,bls_labor_49f6980e,quarterly,"[1999-10-01,2000-01-01)",None,1999Q4
5,demographics,Demographics,usa,labor,Extract from the Quarterly Census of Emploment...,Shoreline clipped TIGER/Line boundaries. More ...,carto-do-public-data.usa_carto.geography_usa_c...,Counties (2018) - shoreline clipped,carto-do-public-data.usa_bls.demographics_labo...,True,eng,BLS QCEW data at county level (2000Q1),usa_bls,USA Bureau of Labor Statistics,bls_labor_c0c1085b,quarterly,"[2000-01-01,2000-04-01)",None,2000Q1
6,demographics,Demographics,usa,labor,Extract from the Quarterly Census of Emploment...,Shoreline clipped TIGER/Line boundaries. More ...,carto-do-public-data.usa_carto.geography_usa_c...,Counties (2018) - shoreline clipped,carto-do-public-data.usa_bls.demographics_labo...,True,eng,BLS QCEW data at county level (2000Q2),usa_bls,USA Bureau of Labor Statistics,bls_labor_59c859e1,quarterly,"[2000-04-01,2000-07-01)",None,2000Q2
7,demographics,Demographics,usa,labor,Extract from the Quarterly Census of Emploment...,Shoreline clipped TIGER/Line boundaries. More ...,carto-do-public-data.usa_carto.geography_usa_c...,Counties (2018) - shoreline clipped,carto-do-public-data.usa_bls.demographics_labo...,True,eng,BLS QCEW data at county level (2000Q3),usa_bls,USA Bureau of Labor Statistics,bls_labor_2ecf6977,quarterly,"[2000-07-01,2000-10-01)",None,2000Q3
8,demographics,Demographics,usa,labor,Extract from the Quarterly Census of Emploment...,Shoreline clipped TIGER/Line boundaries. More ...,carto-do-public-data.usa_carto.geography_usa_c...,Counties (2018) - shoreline clipped,carto-do-public-data.usa_bls.demographics_labo...,True,eng,BLS QCEW data at county

Nice! Let's take the first one (`acs_sociodemogr_b758e778`) that has aggregated data from 2013 to 2018 and check which of its variables have data about the total population.

In [9]:
from cartoframes.data.observatory import Dataset

dataset = Dataset.get('acs_sociodemogr_b758e778')
variables_df = dataset.variables.to_dataframe()
variables_df[variables_df['description'].str.contains('total population', case=False, na=False)]

,agg_method,column_name,dataset_id,db_type,description,id,name,slug,starred,variable_group_id
117,AVG,income_per_capita,carto-do-public-data.usa_acs.demographics_soci...,FLOAT,Per Capita Income in the past 12 Months. Per c...,carto-do-public-data.usa_acs.demographics_soci...,income_per_capita,income_per_capi_8a9352e0,None,None
131,SUM,total_pop,carto-do-public-data.usa_acs.demographics_soci...,FLOAT,Total Population. The total number of all peop...,carto-do-public-data.usa_acs.demographics_soci...,total_pop,total_pop_3cf008b3,None,None


We can see the variable that contains the total population is the one with the slug `total_pop_3cf008b3`. Now we are ready to enrich our areas of influence with that variable.

In [10]:
from cartoframes.data.observatory import Variable
from cartoframes.data.observatory import Enrichment

variable = Variable.get('total_pop_3cf008b3')

isochrones_gdf = Enrichment().enrich_polygons(isochrones_gdf, [variable])
isochrones_gdf.head()

,source_id,data_range,the_geom,total_pop
0,0,900,"MULTIPOLYGON (((-73.94683 40.67001, -73.94623 ...",39526.174107
1,1,900,"MULTIPOLYGON (((-73.97017 40.57732, -73.96957 ...",32932.151177
2,2,900,"MULTIPOLYGON (((-73.99781 40.62418, -73.99755 ...",30933.243102
3,3,900,"MULTIPOLYGON (((-74.03678 40.63431, -74.03618 ...",29436.477749
4,4,900,"MULTIPOLYGON (((-74.00451 40.59723, -74.00391 ...",5902.638057


Great! Let's see the result on a map:

In [11]:
from cartoframes.viz import color_continuous_style

Map([
  Layer(isochrones_gdf, color_continuous_style('total_pop'), title='Total Population'),
  Layer(stores_gdf, size_continuous_style('revenue', size_range=[10,40]), title='Annual Revenue ($)') 
])

At this stage, we could say that the store on the right performs better than others because its area of influence is the one with the lowest population but the store is not the one with lowest revenue. This insight will help us to focus on them in further analyses.

To learn more about discovering the data you want, check out the [data discovery guide](/developers/cartoframes/guides/Data-discovery). To learn more about enriching your data check out the [data enrichment guide](/developers/cartoframes/guides/Data-enrichment/).

### Publish and share your results

The final step in the workflow is to share this interactive map with your colleagues so they can explore the information on their own. Let's do it!

Let's visualize them and add widgets to them so people are able to see some graphs of the information and filter it. To do this, we only have to add `default_widget=True` to the layers.

In [12]:
result_map = Map([
    Layer(
        isochrones_gdf,
        color_continuous_style('total_pop', stroke_width=0, opacity=0.7),
        title='Total Population',
        default_widget=True
    ),
    Layer(
        stores_gdf,
        size_continuous_style('revenue', size_range=[10,40], stroke_color='white'),
        title='Annual Revenue ($)',
        default_widget=True
    ) 
])
result_map

Cool! Now that you have a small dashboard to play with, let's publish it on CARTO so you are able to share it with anyone. To do this, you just need to call the publish method from the Map class:

In [14]:
result_map.publish('starbucks_analysis', password=None, if_exists='replace')

{'id': '94e546ac-9aab-4857-a9b2-46874299c2ed',
 'url': 'https://cartoframes-org.carto.com/u/cartoframes/kuviz/94e546ac-9aab-4857-a9b2-46874299c2ed',
 'name': 'starbucks_analysis',
 'privacy': 'public'}

In order to improve the performance and reduce the size of your map, we recommend to upload the data to CARTO and use the table names in the layers instead. To upload your data, you just need to call `to_carto` with your GeoDataFrame:

In [ ]:
from cartoframes import to_carto

to_carto(stores_gdf, 'starbucks_stores', if_exists='replace')
to_carto(isochrones_gdf, 'starbucks_isochrones', if_exists='replace')

### Conclusion

Congratulations! You have finished this guide and have a sense about how CARTOframes can speed up your workflow. To continue learning, you can check out a specific [Guide](/developers/cartoframes/guides), the [Reference](/developers/cartoframes/reference) to know everything about a class or a method or check the [Examples](/developers/cartoframes/examples) to see CARTOframes in action.